In [3]:
import random

from utils.preprocess import load
from utils.markov_chain import MarkovChain

In [4]:
lines = load('texts/voina_i_mir.txt')
chain = MarkovChain(seqlen=6)

In [5]:
chain.train(lines)

In [6]:
prefix = "в голове "
evals = [(i, chain.inference(prefix=prefix)) for i in range(20)]

In [7]:
evals

[(0, 'в голове послали бы ты его эскадрона т.'),
 (1, 'в голове десяти.'),
 (2, 'в голове пети.'),
 (3, 'в голове нет там в тупик.'),
 (4, 'в голове которой он слишком.'),
 (5, 'в голове ясно в интересовал в обращаясь к.'),
 (6, 'в голове три а ты не помня как алексеич.'),
 (7, 'в голове положением воли это начинать этого.'),
 (8, 'в голове его оживленные сучьями молчал выговорив.'),
 (9, 'в голове пары со шпорами окружающимися глаза.'),
 (10, 'в голове его списку к козырька по против навстречу с.'),
 (11, 'в голове вся на столько не давал.'),
 (12, 'в голове толкнуло в голосом и при князь.'),
 (13, 'в голове пети.'),
 (14, 'в голове усталось так гусарами на мать кавардачку.'),
 (15, 'в голове пьер блестящие штыки и разнообразные.'),
 (16, 'в голове этого тянулись дальше к москве также приятностей'),
 (17, 'в голове и торжественного майор перед гусар.'),
 (18, 'в голове усталые руки прибавил.'),
 (19, 'в голове его мерой хорош солнцем.')]

In [8]:
from utils.preprocess import alphabet
import numpy as np

In [9]:
char2index, index2char = alphabet(lines)
text = ' '.join(lines)
symbols_len = len(char2index)

In [10]:
len(lines)

63239

In [11]:
SEQ_LEN = 30
WINDOW_STEP = 1

In [12]:
inputs = []
targets = []

for i in range(0, len(text) - SEQ_LEN, WINDOW_STEP):
    inputs.append(text[i: i + SEQ_LEN])
    targets.append(text[i + SEQ_LEN])

In [29]:
len(inputs[0]), inputs[0]

(30, 'окончив расспросы и выпытав со')

In [13]:
def encode_line(seq):
    x = np.zeros((len(seq), symbols_len))
    # encode sequence
    for j, char in enumerate(seq):
        x[j, char2index[char]] = 1
    return list(x)

def decode_line(seq):
    result = []
    for j in seq:
        char_index = np.argmax(j)
        char = index2char[char_index]
        result.append(char)
    result = ''.join(result)
    return result

In [14]:
decode_line(encode_line('кес'))

'кес'

In [16]:
X = []
Y = []

for i, seq in enumerate(inputs):
    x = encode_line(seq)
    y = encode_line(targets[i])[0]
    X.append(x)
    Y.append(y)

In [39]:
all_data = list(zip(X, Y))
all_data = random.choices(all_data, k=500_000)
X_tmp, Y_tmp = list(zip(*all_data))
X_data, Y_data = np.array(X_tmp), np.array(Y_tmp)

In [40]:
X_data[0]

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [41]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, test_size=0.2)

In [42]:
X_train.shape, y_train.shape

((400000, 30, 34), (400000, 34))

In [43]:
import tensorflow as tf
lstm = tf.keras.models.Sequential()
lstm.add(
    tf.keras.layers.LSTM(
        128,
        input_shape=(SEQ_LEN, symbols_len),
        dropout=0.3,
        recurrent_dropout=0.3,
    )
)
lstm.add(tf.keras.layers.Dense(symbols_len, activation='softmax'))
lstm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
     

In [44]:
lstm.fit(X_train, y_train, 
        epochs=10, 
        validation_data=(X_test, y_test),
        use_multiprocessing=True,
        workers=-1
)

Epoch 1/10
12500/12500 [==============================] - 297s 24ms/step - loss: 2.4314 - accuracy: 0.2864 - val_loss: 2.1183 - val_accuracy: 0.3598
Epoch 2/10
12500/12500 [==============================] - 297s 24ms/step - loss: 2.1532 - accuracy: 0.3495 - val_loss: 1.9690 - val_accuracy: 0.3957
Epoch 3/10
12500/12500 [==============================] - 266s 21ms/step - loss: 2.0587 - accuracy: 0.3742 - val_loss: 1.8951 - val_accuracy: 0.4162
Epoch 4/10
12500/12500 [==============================] - 330s 26ms/step - loss: 2.0054 - accuracy: 0.3899 - val_loss: 1.8564 - val_accuracy: 0.4282
Epoch 5/10
12500/12500 [==============================] - 199s 16ms/step - loss: 1.9705 - accuracy: 0.3985 - val_loss: 1.8194 - val_accuracy: 0.4379
Epoch 6/10
12500/12500 [==============================] - 196s 16ms/step - loss: 1.9465 - accuracy: 0.4034 - val_loss: 1.7911 - val_accuracy: 0.4435
Epoch 7/10
12500/12500 [==============================] - 209s 17ms/step - loss: 1.9295 - accuracy: 0.4080

In [45]:
def sample(predictions):
    predictions = np.log(predictions)
    exp_predictions = np.exp(predictions)
    predictions = exp_predictions / np.sum(exp_predictions)
    probabilities = np.random.multinomial(1, predictions, 1)
    return np.argmax(probabilities)

def lstm_predict(model, string, max_len=None):
    print(string, end="")
    result = list(string)
    string = encode_line(string)
    while True:
        prediction = model.predict(np.array([string]))
        # ind = sample(prediction[0])
        # ind = np.argmax(prediction[0])
        # char = index2char[ind]
        char = random.choices(index2char, weights=prediction[0])[0]
        string.append(encode_line(char)[0])
        result.append(char)
        if (char == '.' and not max_len) or (max_len and len(result) > max_len):
            break
        
    return ''.join(result)

In [55]:
example = "вместо тысящи слов "
lstm_predict(lstm, example)

1/1 [==============================] - 0s 17ms/step


'вместо тысящи слов городил.'

In [60]:
prefixes = [
    "жить во ",
    "по лицу ",
    "смысл жизни в том ",
    "фактически ",
    "голос его был "
]

In [61]:
result = []
for n, pref in enumerate(prefixes):
    result.append((
        n,
        ("markov: " + chain.inference(pref)),
        ("lstm: " + lstm_predict(lstm, pref))
    ))
    # print("markov:", chain.inference(pref, max_length=80))
    # print("lstm:", lstm_predict(lstm, pref))

1/1 [==============================] - 0s 22ms/step


In [62]:
for t in result:
    print(t[0])
    print(t[1])
    print(t[2])

0
markov: жить во всем существования невольно здоровлениями и
lstm: жить во взглядык которою наражала за перать.
1
markov: по лицу сорок.
lstm: по лицу расинердосяно чтося обратя такта желал этого вся сотрещают приказовали народ готов что он ились любился и пьер дображному нибедь должно и вчем своей в йску.
2
markov: смысл жизни в том что.
lstm: смысл жизни в том частроной большу.
3
markov: фактически от нее прелестная кровь лилась тяжело и
lstm: фактически кригами аспровицла спольна к его должалось же самое вперед обораа.
4
markov: голос его был высоко ценить свое имя имя которые казаками
lstm: голос его был на французских отдушал.


In [63]:
lstm_predict(lstm, 'мне нравится как говорят киты ', max_len=300)

1/1 [==============================] - 0s 35ms/step


'мне нравится как говорят киты и будьт огнянувшегося что он глуду графино рапиними же не должно это продля белое государь и местром умянное вой то же. он других бою слушавшись на на. было марья. общись. княжна и же это звечилох несмотрели французов. так чтобы мне подвыстить. ростов. я он извесандни фр'